<a href="https://colab.research.google.com/github/khinan/folium_project_n3ts/blob/main/Khinanti_10_1_Benda_Baru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import folium
import pandas as pd
import geopandas as gpd
from google.colab import drive


drive.mount('/content/drive')
indonesia_admin = gpd.read_file('/content/drive/MyDrive/ADMIN_INDO/ADMINISTRASI_AR_DESAKEL_10K.shp')

aoi_admin = indonesia_admin[indonesia_admin['WADMKK'] == 'Benda Baru']

csv_path = '/content/drive/MyDrive/tangsel_kelurahan_KK.csv'

admin_gdf = aoi_admin

csv_data = pd.read_csv(csv_path, header = 2, delimiter= ';')
m = folium.Map(location = [-6.2, 106.6], zoom_start=11)
choropleth = folium.Choropleth(
    geo_data=admin_gdf,
    name='choropleth',
    data=admin_gdf,
    columns=['Benda Baru', '13513'],
    key_on= "feature.properties.NAMOBJ",
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Jumlah KK'
).add_to(m)

jumlah_KK = folium.GeoJson(
    data=merged_gdf,
    name='Jumlah KK',
    style_function=lambda feature: {'fillOppacity': 0, 'color': 'transparent'},
    popup= folium.GeoJsonPopup(
        fields=['Benda Baru', '13513'],
        aliases=['Benda Baru', '13513'],

    )
).add_to(chorolopeth)


folium.LayerControl().add_to(m)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KeyError: "None of ['Kelurahan'] are in the columns"